In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import time

In [2]:
def pps(n,ND) :
    P=n*100/ND
    PP= "{:.2f}".format(P)
    print("\r",end="")
    print("processes",PP,"% completed:   ",end="")
    i=50*n/ND
    a=50-int(i)
    b=(int(i)+1)*"🟩"
    c=(a-1)*"⬜️"
    print(b,end="")
    print(c, end="")
    if n==ND:
        print("\r","computing successfully completed!   ",50*"🟦")

In [3]:
def initial_condition(I0):
    G=nx.read_gpickle('Graph.gpickle')
    for n_0 in G.nodes:
        G.nodes[n_0]['x']='s'
        G.nodes[n_0]['tpe']= 0
        G.nodes[n_0]['tpq']= 0
        G.nodes[n_0]['Q']=False
        if random.random()<I0:
            G.nodes[n_0]['x']='i'
    return G

In [4]:
def Count_nodes(Graph):
    qnode=[]
    snode=[]
    enode=[]
    inode=[]
    rnode=[]
    vnode=[]
    for n_1 in Graph.nodes:
        if Graph.nodes[n_1]['Q']==True:
            qnode.append(n_1)
        if Graph.nodes[n_1]['x']=='s':
            snode.append(n_1)
        if Graph.nodes[n_1]['x']=='e':
            enode.append(n_1)  
        if Graph.nodes[n_1]['x']=='i':
            inode.append(n_1)  
        if Graph.nodes[n_1]['x']=='r':
            rnode.append(n_1)
    data=[]
    data.append(len(snode))
    data.append(len(qnode))
    data.append(len(enode))
    data.append(len(inode))
    data.append(len(rnode))
    return (data,inode)

In [5]:
def SEIR_dynamic(I0,beta,gamma,t_e,t_q,P2):

    G = initial_condition(I0)

    s_all=[]
    sdata=[]
    qdata=[]
    edata=[]
    idata=[]
    rdata=[]

    while len(rdata)<20 or rdata[-1]!=rdata[-3]:

        (data,inode)= Count_nodes(Graph=G)

        sdata.append(data[0])
        qdata.append(data[1])
        edata.append(data[2])
        idata.append(data[3])
        rdata.append(data[4])

        for i in G.nodes:
            if G.nodes[i]['x']=='e':
                if G.nodes[i]['tpe']>=np.random.poisson(t_e,size=1)[0]:
                    G.nodes[i]['x']='i'
                else:
                    G.nodes[i]['tpe']+=1
            if G.nodes[i]['Q']==True :
                if G.nodes[i]['tpq']>=np.random.poisson(t_q,size=1)[0]:
                    G.nodes[i]['Q']=False
                else:
                    G.nodes[i]['tpq']+=1
            if (i in inode) and G.nodes[i]['x']=='i':
                for n_5 in G.adj[i]:
                    if ((G.nodes[n_5]['x']=='s') and (random.random()<P2)):
                        G.nodes[n_5]['Q']=True
                        G.nodes[n_5]['tpq']=0
                    if G.nodes[n_5]['x']=='s':
                        if ((G.nodes[i]['Q']==False) and (random.random()<beta)):
                            G.nodes[n_5]['x']='e'
                        if ((G.nodes[i]['Q']==True) and (random.random()<beta*0.5)):
                            G.nodes[n_5]['x']='e'
                if random.random()<gamma:
                    G.nodes[i]['x']='r'
                    G.nodes[i]['Q']=False

    RES=[sdata,edata,idata,rdata,qdata]
    return RES

In [6]:
a=time.time()
data=SEIR_dynamic(I0= 0.01,beta= 0.8,gamma= 1,t_e= 5,t_q= 6,P2= 0.3)
b=time.time()
b-a

0.3990023136138916

In [7]:
plt.figure(figsize=(15,7))
plt.plot(data[0], 'darkblue', label='Susceptibles')
plt.plot(data[1],'y', label='Exposeds')
plt.plot(data[2],'brown', label='Infectious')
plt.plot(data[3], 'darkgreen', label='Recovereds')
plt.plot(data[4], 'k', label='Isolated')
plt.legend(loc=0)
plt.xlabel('Time(day)')
plt.ylabel('people number')
plt.grid(color = 'navy', linestyle = '--', linewidth = 0.5)
plt.savefig('SEIR-Network(2-0)')
plt.show()

In [8]:
plt.figure(figsize=(15,7))
#plt.plot(data[0], 'darkblue', label='Susceptibles')
plt.plot(data[1],'y', label='Exposeds')
plt.plot(data[2],'brown', label='Infectious')
#plt.plot(data[3], 'darkgreen', label='Recovereds')
plt.plot(data[4], 'k', label='Isolated')
plt.legend(loc=0)
plt.xlabel('Time(day)')
plt.ylabel('people number')
plt.grid(color = 'navy', linestyle = '--', linewidth = 0.5)
#plt.savefig('SEIR-Network(2-0)')
plt.show()